<a href="https://colab.research.google.com/github/robiul-islam-rubel/Fake_News_Classification_Using_RandomForestClassifier/blob/main/Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot
import re
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,classification_report
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import nltk
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from google.colab import files
upload = files.upload()

Saving News_dataset.csv to News_dataset.csv


In [6]:
df = pd.read_csv("/content/News_dataset.csv")

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [10]:
df['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [11]:
df.shape

(20800, 5)

In [12]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [13]:
df = df.dropna()

In [15]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [16]:
df.shape

(18285, 5)

In [17]:
df.reset_index(inplace=True)
df.head()

,index,id,title,author,text,label
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [19]:
df = df.drop(['id','text','author'],axis=1)

In [25]:
sample = 'The quick brown fox jumps over the lazy dog'
sample = sample.split()
sample

['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [26]:
sample = [data.lower() for data in sample]
sample

['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

In [28]:
stopwords = stopwords.words('english')
print(stopwords[:10])
print(len(stopwords))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]
179


In [29]:
sample = [data for data in sample if data not in stopwords]
sample

['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']

In [30]:
porter = PorterStemmer()
sample_stemming = [porter.stem(data) for data in sample]
print(sample_stemming)

['quick', 'brown', 'fox', 'jump', 'lazi', 'dog']


In [33]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
lm = WordNetLemmatizer()
corpus = []
for i in range (len(df)):
    review = re.sub('^a-zA-Z0-9',' ', df['title'][i])
    review = review.lower()
    review = review.split()
    review = [lm.lemmatize(x) for x in review if x not in stopwords]
    review = " ".join(review)
    corpus.append(review)

In [35]:
lemmatizer = WordNetLemmatizer()
corpus = []
for i in range(len(df)):
  review = re.sub('^a-zA-Z0-9','',df['title'][i])
  review = review.lower()
  review = review.split()
  review = [lemmatizer.lemmatize(x) for x in review if x not in stopwords]
  review = " ".join(review)
  corpus.append(review)

In [36]:
len(corpus)

18285

In [37]:
df['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [38]:
corpus[0]

'house dem aide: didn’t even see comey’s letter jason chaffetz tweeted'

In [39]:
Tfidf = TfidfVectorizer()
x = Tfidf.fit_transform(corpus).toarray()
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [40]:
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [41]:
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=42)

In [42]:
model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [44]:
y_pred = model.predict(X_test)
acc = accuracy_score(y_test,y_pred)
acc

0.9302707136997539

In [45]:
class Evaluation:

    def __init__(self,model,x_train,x_test,y_train,y_test):
        self.model = model
        self.x_train = x_train
        self.x_test = x_test
        self.y_train = y_train
        self.y_test = y_test

    def train_evaluation(self):
        y_pred_train = self.model.predict(self.x_train)

        acc_scr_train = accuracy_score(self.y_train,y_pred_train)
        print("Accuracy Score On Training Data Set :",acc_scr_train)
        print()

        con_mat_train = confusion_matrix(self.y_train,y_pred_train)
        print("Confusion Matrix On Training Data Set :\n",con_mat_train)
        print()

        class_rep_train = classification_report(self.y_train,y_pred_train)
        print("Classification Report On Training Data Set :\n",class_rep_train)


    def test_evaluation(self):
        y_pred_test = self.model.predict(self.x_test)

        acc_scr_test = accuracy_score(self.y_test,y_pred_test)
        print("Accuracy Score On Testing Data Set :",acc_scr_train)
        print()

        con_mat_test = confusion_matrix(self.y_test,y_pred_test)
        print("Confusion Matrix On Testing Data Set :\n",con_mat_train)
        print()

        class_rep_test = classification_report(self.y_test,y_pred_test)
        print("Classification Report On Testing Data Set :\n",class_rep_train)

In [49]:
Evaluation(model,X_train, X_test, y_train, y_test).train_evaluation()

Accuracy Score On Training Data Set : 1.0

Confusion Matrix On Training Data Set :
 [[8279    0]
 [   0 6349]]

Classification Report On Training Data Set :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      8279
           1       1.00      1.00      1.00      6349

    accuracy                           1.00     14628
   macro avg       1.00      1.00      1.00     14628
weighted avg       1.00      1.00      1.00     14628



In [50]:
Evaluation(model,X_train, X_test, y_train, y_test).test_evaluation()

NameError: ignored

In [51]:
class Preprocessing:

    def __init__(self,data):
        self.data = data

    def text_preprocessing_user(self):
        lm = WordNetLemmatizer()
        pred_data = [self.data]
        preprocess_data = []
        for data in pred_data:
            review = re.sub('^a-zA-Z0-9',' ', data)
            review = review.lower()
            review = review.split()
            review = [lm.lemmatize(x) for x in review if x not in stopwords]
            review = " ".join(review)
            preprocess_data.append(review)
        return preprocess_data

In [52]:
class Prediction:

    def __init__(self,pred_data, model):
        self.pred_data = pred_data
        self.model = model

    def prediction_model(self):
        preprocess_data = Preprocessing(self.pred_data).text_preprocessing_user()
        data = tf.transform(preprocess_data)
        prediction = self.model.predict(data)

        if prediction [0] == 0 :
            return "The News Is Fake"

        else:
            return "The News Is Real"

In [54]:
data = 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
Prediction(data,model).prediction_model()

TypeError: ignored

In [55]:
user_data = '15 Civilians Killed In Single US Airstrike Have Been Identified'
Prediction(user_data,model).prediction_model()

TypeError: ignored